In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/spamemail'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spamemail/spam.csv


BERT¶
BERT, which stands for Bidirectional Encoder Representations from Transformers, is a powerful natural language processing (NLP) model introduced by Google AI researchers in 2018.
BERT is a type of transformer-based model that has had a significant impact on a wide range of NLP tasks, including text classification, named entity recognition, sentiment analysis, and question-answering.

**Import Needed Modules**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix, classification_report

2024-05-15 17:14:36.699757: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 17:14:36.699948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 17:14:36.866108: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
import pandas as pd
filename = "/kaggle/input/spamemail/spam.csv"
df = pd.read_csv(filename)
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


**EDA**

In [13]:
# Read Data
df = pd.read_csv('/kaggle/input/spamemail/spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [14]:
# Data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [15]:
# Output distribution
df['Category'].value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

15% spam emails, 85% ham emails: This indicates class imbalance

**Data Preprocessing**

In [16]:
# Get all spams labels
spam_df = df[df['Category']=='spam']
spam_df.shape

(747, 2)

In [17]:
# Get all hams labels
ham_df = df[df['Category']=='ham']
ham_df.shape

(4825, 2)

**Apply downsample approach**

In [18]:
# Get random sample from ham data in size of spam data
ham_df_downsampled = ham_df.sample(spam_df.shape[0])
ham_df_downsampled.shape

(747, 2)

In [19]:
# Create balanced data that contains the same size of two categories
balanced_df = pd.concat([ham_df_downsampled, spam_df])
balanced_df.shape

(1494, 2)

In [20]:
# New balanced data distribution
balanced_df['Category'].value_counts()

Category
ham     747
spam    747
Name: count, dtype: int64

**Data Encoding**

In [21]:
# Create a new column that contains encoded values of Category ['ham':0, 'spam':1]
balanced_df['spam']= balanced_df['Category'].apply(lambda x: 1 if x=='spam' else 0)
balanced_df.sample(5)

,Category,Message,spam
3176,spam,UR GOING 2 BAHAMAS! CallFREEFONE 08081560665 a...,1
1092,ham,Please da call me any mistake from my side sor...,0
4053,ham,"Ya ok, then had dinner?",0
4676,spam,"Hi babe its Chloe, how r u? I was smashed on s...",1
2015,ham,Ambrith..madurai..met u in arun dha marrge..re...,0


**Now lets import BERT model and get embeding vectors for few sample statements**

In [22]:
# Preprocessing model
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

In [23]:
# BERT model
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [25]:
# This function apply bert preprocessing on sentences
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

**Get embeding vectors for few sample words. Compare them using cosine similarity**

In [26]:
e = get_sentence_embeding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
])

print(cosine_similarity([e[0]],[e[1]]))
print(cosine_similarity([e[2]],[e[3]]))
print(cosine_similarity([e[4]],[e[5]]))

[[0.9911088]]
[[0.84945345]]
[[0.9164165]]


**Values near to 1 means they are similar. 0 means they are very different. Above you can use comparing "banana" vs "grapes" you get 0.99 similarity as they both are fruits**